# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


### Workspace 

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134744
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-134744


### Configure Compute Cluster

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "automl-project"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes =  2)


Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

Citation: Davide Chicco, Giuseppe Jurman: "Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone". BMC Medical Informatics and Decision Making 20, 16 (2020)


Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

**12 clinical features:**

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

Here, we performed Logistic Regression on this Dataset inorder to predict the mortality by heart failure.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.


In [4]:
from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name='heart-disease Dataset')
df = ds.to_pandas_dataframe()
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.


Automated machine learning supports ensemble models. like **voting ensemble** and **stacking ensemble** methods for combining models.

In this project, AutoML rapidly iterates over many combinations of algorithms and hyperparameters, and find out the best classification model based on a **Accuracy** success metric.


In [5]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=3,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [6]:
# TODO: Submit your experiment
run = exp.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(run).show()
# wait for completion
run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_90066085-ec01-4d85-a02e-d9b9050f081f',
 'target': 'automl-project',
 'status': 'Completed',
 'startTimeUtc': '2021-01-15T10:52:34.734581Z',
 'endTimeUtc': '2021-01-15T11:15:28.399693Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'automl-project',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity-capstone","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-134744","workspace_name":"quick-starts-ws-134744","region":"southcentralus","compute_target":"automl-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"n

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
aml_best_run, model = run.get_output()
aml_best_run_metrics = aml_best_run.get_metrics()

print(model)
print('Best Run Id: ', aml_best_run.id)
print('\n Accuracy:', aml_best_run_metrics['accuracy'])

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               max_leaves=0,
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                                                            

In [9]:
import joblib
joblib.dump(model,'outputs/automlmodel.pkl')
print(aml_best_run.get_tags())


{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":4}', 'ensembled_iterations': '[29, 14, 2, 4, 19, 25, 20]', 'ensembled_algorithms': "['LightGBM', 'ExtremeRandomTrees', 'RandomForest', 'RandomForest', 'XGBoostClassifier', 'ExtremeRandomTrees', 'XGBoostClassifier']", 'ensemble_weights': '[0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125]', 'best_individual_pipeline_score': '0.8460942760942761', 'best_individual_iteration': '29', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="outputs/automlmodel.pkl",
                        model_name = "automl")


Registering model automl


In [11]:
%%writefile amlscore.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("automl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Writing amlscore.py


In [12]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
env = Environment.get(workspace=ws, name="AzureML-AutoML")

In [13]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

aciconfig2 = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               enable_app_insights=True, 
                                               auth_enabled=True)



ws = Workspace.from_config()
model = Model(ws, 'automl')

inference_config = InferenceConfig(entry_script="amlscore.py", environment=env)

service_name = 'aml-service'
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config= aciconfig2)

service.wait_for_deployment(show_output=True)



print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [14]:
print(service.get_logs())

2021-01-15T11:26:54,433852921+00:00 - gunicorn/run 
2021-01-15T11:26:54,437092234+00:00 - nginx/run 
2021-01-15T11:26:54,437155434+00:00 - rsyslog/run 
2021-01-15T11:26:54,437686336+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [15]:
print("scoring URI: " + service.scoring_uri)
print("Swagger URI: " + service.swagger_uri)
print("Authetication Key: " + service.get_keys()[0])

scoring URI: http://ab113c9e-510a-4556-bb40-4c9317d9e711.southcentralus.azurecontainer.io/score
Swagger URI: http://ab113c9e-510a-4556-bb40-4c9317d9e711.southcentralus.azurecontainer.io/swagger.json
Authetication Key: YvSHq1nEfQa9Kgz4LQKHAhez5CKy9r77


In [16]:
primary, secondary = service.get_keys()
print(primary,secondary)

YvSHq1nEfQa9Kgz4LQKHAhez5CKy9r77 AgW32x4pfddUMecEwBaTPGgKeLvWTlxO


TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
import requests
import json
scoringuri = service.scoring_uri
key=primary
data= { "data":
       [
           {
               'age': 50,
               'anaemia': 255,
               'creatinine_phosphokinase': 0,
               'diabetes': 0,
               'ejection_fraction': 83,
               'high_blood_pressure': 1,
               'platelets': 263000,
               'serum_creatinine': 40,
               'serum_sodium':140,
               'sex':1,
               'smoking':1,
               'time':4
               
               },
            {
               'age': 55,
               'anaemia': 245,
               'creatinine_phosphokinase': 1,
               'diabetes': 1,
               'ejection_fraction': 38,
               'high_blood_pressure': 1,
               'platelets': 167300,
               'serum_creatinine': 50,
               'serum_sodium':105,
               'sex':1,
               'smoking':1,
               'time':6
            
               
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

[1, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()


# load existing web service
service = Webservice(name="aml-service", workspace=ws)
service.update(enable_app_insights=True)
logs = service.get_logs()

print(logs)

2021-01-15T11:26:54,433852921+00:00 - gunicorn/run 
2021-01-15T11:26:54,437092234+00:00 - nginx/run 
2021-01-15T11:26:54,437155434+00:00 - rsyslog/run 
2021-01-15T11:26:54,437686336+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [23]:
service.delete()
print(service)


AciWebservice(workspace=Workspace.create(name='quick-starts-ws-134744', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-134744'), name=aml-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Deleting, tags=http://ab113c9e-510a-4556-bb40-4c9317d9e711.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})


In [24]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

